In [1]:
#Website untersuchen
import requests
from bs4 import BeautifulSoup

#Tabelle bauen
import pandas as pd

#Bilder organisieren
import os
import urllib.request

In [2]:
#dataframe erzeugen
index=['Titel','Beschreibung','Datum','Seite/Ausgabe','ID','Register','Themen','Personen','Ort','Bild']
pers_lst=[]

In [3]:
def ScrapItem(baseURL):
    title=""
    personen=[]
    themen=[]
    orte=[]
    bildURL=""
    response = requests.get(baseURL)
    # BeautifulSoup HTML-Dokument aus dem Quelltext parsen
    html = BeautifulSoup(response.content, 'html.parser')#text
    print("==========================")
    title = html.find('h1').getText()
    print("Titel gefunden: " + title)
    #Beschreibung
    beschreibung = html.find_all("section", {"class": "border-bottom"})
    beschreibung=str(beschreibung)
    b_anfang=beschreibung.find("/strong>",5,len(beschreibung))+9
    b_ende=beschreibung.find("</p>",b_anfang,len(beschreibung))
    beschreibung=beschreibung[b_anfang:b_ende]
    print("Beschreibung gefunden: " + beschreibung)
    #Metadatenblock
    X = html.find('h1').nextSibling.nextSibling
    X=str(X)
    #print(X)
    #Datum
    datum_start = X.find("vom",0,len(X))+4
    datum_end = X.find(",",datum_start,len(X))
    datum = X[datum_start:datum_end]
    print("Datum gefunden: " + datum)
    #Seite/Ausgabe
    s_anfang = X.find(",",datum_end,len(X))+2
    s_end = X.find(";",s_anfang,len(X))
    seite = X[s_anfang:s_end]
    print("Seite/Ausgabe gefunden: " + seite)
    #ID
    id_anfang = X.find("ID:",s_end,len(X))+4
    id_end = X.find("<",id_anfang,len(X))
    did = X[id_anfang:id_end]
    print("ID gefunden: " + did)
    #Register
    register_anfang = X.find("(",id_end,len(X))+1
    register_end = X.find(")",register_anfang,len(X))
    register = X[register_anfang:register_end]
    print("Register gefunden: " + register)
    L= html.find_all("a")
    #Alle relevanten Links:
    for l in range(len(L)):
        currentLink= str(L[l]['href'])
        currentLinkText = str(L[l].string)
        #print(currentLinkText + '|' + currentLink)
        #themen-artikel
        if currentLink != None and "themen-artikel" in currentLink:
            print("Thema gefunden: " + currentLinkText)
            themen.append(currentLinkText)
        #Ll_themen = 
        #personen-artikel
        if currentLink != None and "personen-artikel" in currentLink:
            print("Person gefunden: " + currentLinkText)
            personen.append(currentLinkText)
            pers_lst.append(currentLinkText)
        #asp?Ort
        if currentLink != None and "asp?Ort" in currentLink:
            print("Ort gefunden: " + currentLinkText)
            orte.append(currentLinkText)
        #tif.large.jpg
        if currentLink != None and "tif.large.jpg" in currentLink:
            print("Bild gefunden: " + currentLink)
            #https://hait.tu-dresden.de/slub/1931/01/files/27/files/00000007.tif.large.jpg
            bildURL="https://hait.tu-dresden.de/"+currentLink
            #print(bildURL)
    lst_item=[]
    lst_item.append(title)
    lst_item.append(beschreibung)
    lst_item.append(datum)
    lst_item.append(seite)
    lst_item.append(did)
    lst_item.append(register)
    lst_item.append(", ".join(themen))
    lst_item.append(", ".join(personen))
    lst_item.append(", ".join(orte))
    lst_item.append(bildURL)

    #print(lst_item)
    return lst_item



In [4]:
s1 = pd.Series([])
pre_lst=[]
f = open("RadebergLinks.txt", "r")
for x in f:
    x=x.strip()
    x=x[1:len(x)-1]
    item = ScrapItem(x)
    pre_lst.append(item)
    
    #print(x)



/tmp/ipykernel_1273/3556134726.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s1 = pd.Series([])


Titel gefunden: In Radeberg sollte Blut fließen!
Beschreibung gefunden: OG-Treffen der Nationalsozialisten in Radeberg führte zu Zusammenstößen mit Marxisten.
Datum gefunden: 27.01.1931
Seite/Ausgabe gefunden: S. 7 (Ausgabe: Gau Sachsen)
ID gefunden: 1475
Register gefunden: SLUB Dresden, 1. Mifi.Z.126,2
Thema gefunden: SPD
Thema gefunden: NSDAP öffentliche Veranstaltungen regional/lokal
Person gefunden: Möckel (OGL Radeberg)
Ort gefunden: Radeberg
Bild gefunden: ../../slub/1931/01/files/27/files/00000007.tif.large.jpg
Titel gefunden: SA. marschiert... Werbemarsch und Besichtigung des Sturmbannes 1
Beschreibung gefunden: Bericht des neu aufgestellten Sturmbanners 1 (Dresden rechts der Elbe).

// Erschienen in der Beilage "Von der Kampf-Front".

// Autor: Harry Brückner.

// Weitere genannte Personen: Hill (Sturmführer Sturm 21 aus Radeberg).
Datum gefunden: 22.04.1931
Seite/Ausgabe gefunden: S. 6 (Ausgabe: Gau Sachsen)
ID gefunden: 1830
Register gefunden: SLUB Dresden, 1. Mifi.Z.126,2
T

In [5]:
#df()

In [11]:
#Daten zu Excel
#print(len(pre_lst))
#print(pre_lst)
df = pd.DataFrame(pre_lst, columns = index)
df.to_excel(r'Radeberg_HAIT.xlsx', index=False)
#print(df)
df.head()
#df.tail()

,Titel,Beschreibung,Datum,Seite/Ausgabe,ID,Register,Themen,Personen,Ort,Bild
0,In Radeberg sollte Blut fließen!,OG-Treffen der Nationalsozialisten in Radeberg...,27.01.1931,S. 7 (Ausgabe: Gau Sachsen),1475,"SLUB Dresden, 1. Mifi.Z.126,2","SPD, NSDAP öffentliche Veranstaltungen regiona...",Möckel (OGL Radeberg),Radeberg,https://hait.tu-dresden.de/../../slub/1931/01/...
1,SA. marschiert... Werbemarsch und Besichtigung...,Bericht des neu aufgestellten Sturmbanners 1 (...,22.04.1931,S. 6 (Ausgabe: Gau Sachsen),1830,"SLUB Dresden, 1. Mifi.Z.126,2","NSDAP Gau Sachsen, SA/SS (inkl. Stahlhelm, DLV)","Bennecke, Heinrich (Dr.), Seifert, Helmut (Sei...","Radeberg, Dresden",https://hait.tu-dresden.de/../../slub/1931/04/...
2,SA. marschiert in Königsbrück!_Protest der ost...,Bericht über eine politische Kundgebung am 27....,01.07.1931,S. 6–7 (Ausgabe: Gau Sachsen),2456,"SLUB Dresden, 1. Mifi.Z.126,3",NSDAP öffentliche Veranstaltungen regional/lok...,"Tschammer und Osten, Hans von (SA, Sportkomm.)...","Radeburg, Radeberg, Bautzen, Königsbrück, Dres...",https://hait.tu-dresden.de/../../slub/1931/07/...
3,Sozialdemokratische Denunzianten_Rote Bonzen s...,Bericht über eine Klage des Radeberger Stadtve...,13.08.1931,S. 1 (Ausgabe: Gau Sachsen),2926,"SLUB Dresden, 1. Mifi.Z.126,3","Verfahren gegen NSDAP/Parteigenossen, Angriffe...","Kluge, Rudolf (Dr., Bürgermeister DD), Leupolt...",Radeberg,https://hait.tu-dresden.de/../../slub/1931/08/...
4,Zinswucher!_1810 Prozent Jahreszinsen vom Fina...,Polemik gegen das Finanzamt Radeberg i. Sa.,13.08.1931,S. 3 (Ausgabe: Gau Sachsen),2928,"SLUB Dresden, 1. Mifi.Z.126,3",Angriffe gegen die Verwaltung,,Radeberg,https://hait.tu-dresden.de/../../slub/1931/08/...
